<a href="https://colab.research.google.com/github/icecat14159/PL-Repo./blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80HW01_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   從GoogleSheet讀入欄位資料
2.   讓使用者操作各種功能，並將數據寫入原表單




---


GoogleSheet:
https://docs.google.com/spreadsheets/d/1LjwO1L25McUPX4KpWXlketCnyb5kcBYTINDzHYnjdq8/edit?usp=sharing

In [46]:
#前置作業
#驗證帳號並建立 gspread client（Colab 會跳出 OAuth 流程）
from google.colab import auth
auth.authenticate_user() #會要求選 Google 帳戶授權

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds) #gspread client，後續用它操作試算表

In [25]:
#開啟試算表（用名稱、或用 URL/KEY）
SPREADSHEET_NAME = "程式語言01-日常支出速算與分攤"
sh = gc.open(SPREADSHEET_NAME)   #以名稱開啟

# 若要用 URL/key 開啟，使用下列方法：

# SPREADSHEET_URL = "放上你的試算表連結"
# spreadsheet_key = SPREADSHEET_URL.split("/d/")[1].split("/")[0]
# sh = gc.open_by_key(spreadsheet_key)

ws = sh.sheet1 #第一個工作表；或使用 sh.worksheet("工作表名稱")

In [26]:
#讀取試算表為 pandas.DataFrame
import pandas as pd
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# header=0 表示試算表第一列為欄位名稱；evaluate_formulas=True 取得計算後的值
df = get_as_dataframe(ws, header=0, evaluate_formulas=True)

# 清理空白列與可能的 Unnamed index 欄
df = df.dropna(how="all").reset_index(drop=True)
if any(col.startswith("Unnamed") for col in df.columns):
    df = df.loc[:, ~df.columns.str.startswith("Unnamed")]

df.head()

,日期,時間,項目,金額,備註
0,2025/9/11,8:45,早餐,90.0,NaN
1,2025/9/11,13:01,午餐,100.0,NaN
2,2025/9/12,15:30,文具,16.0,西卡紙


In [44]:
#記帳功能
from datetime import datetime, timezone, timedelta

def add_record(ws):
    # 1. 取得非空白的「項目」
    item = input("請輸入消費項目（例如：午餐、交通）: ").strip()
    while not item:
        item = input("項目不可空白，請重新輸入: ").strip()

    # 2. 取得並驗證「金額」為數字
    while True:
        amt_raw = input("請輸入金額（只接受數字）: ").strip()
        try:
            amount = float(amt_raw)
            break
        except ValueError:
            print("金額格式錯誤，請輸入數字。")

    # 3. 詢問是否加入備註
    note = ""
    yn = input("是否加入備註？(y/n): ").strip().lower()
    if yn == "y":
        note = input("請輸入備註: ").strip()

    # 4. 取得台北時區現在的日期與時間（UTC+8）
    now = datetime.now(timezone(timedelta(hours=8)))
    date_str = now.strftime("%Y/%m/%d")
    time_str = now.strftime("%H:%M")

    # 5. 組成列並寫入（欄位順序：日期、時間、項目、金額、備註）
    row = [date_str, time_str, item, amount, note]

    try:
        ws.append_row(row, value_input_option='USER_ENTERED')
        print("已寫入：", row)
    except Exception as e:
        print("寫入失敗：", e)

In [35]:
#刪除功能
import pandas as pd
from gspread_dataframe import get_as_dataframe

def delete_record(ws):
    # 抓資料並顯示
    df = get_as_dataframe(ws, header=0, evaluate_formulas=True).dropna(how="all").reset_index(drop=True)
    if df.empty:
        print("目前沒有資料。")
        return

    print("目前資料：")
    print(df.reset_index().rename(columns={"index":"編號"}))  # 加上編號方便選擇

    # 輸入要刪除的編號
    while True:
        try:
            idx = int(input("請輸入要刪除的編號: "))
            if 0 <= idx < len(df):
                break
            else:
                print("超出範圍。")
        except ValueError:
            print("請輸入數字。")

    # gspread 的 delete_row 用「工作表的第幾列」，要加上標題列（所以 +2）
    ws.delete_rows(idx + 2)
    print("已刪除第", idx, "筆資料。")

In [43]:
#修改功能
def modify_record(ws):
    # 抓資料並顯示
    df = get_as_dataframe(ws, header=0, evaluate_formulas=True).dropna(how="all").reset_index(drop=True)
    if df.empty:
        print("目前沒有資料。")
        return

    print("目前資料：")
    print(df.reset_index().rename(columns={"index":"編號"}))

    # 輸入要修改的編號
    while True:
        try:
            idx = int(input("請輸入要修改的編號: "))
            if 0 <= idx < len(df):
                break
            else:
                print("超出範圍。")
        except ValueError:
            print("請輸入數字。")

    # 讀取原本資料
    old_row = df.loc[idx].to_dict()
    print("原資料：", old_row)

    # 逐欄修改（若輸入空白，保留原值）
    new_item = input(f"項目 [{old_row['項目']}]: ").strip() or old_row['項目']
    new_amount_raw = input(f"金額 [{old_row['金額']}]: ").strip()
    new_amount = float(new_amount_raw) if new_amount_raw else old_row['金額']
    new_note = input(f"備註 [{old_row['備註']}]: ").strip() or old_row['備註']

    # 日期與時間更新為現在
    now = datetime.now(timezone(timedelta(hours=8)))
    date_str = now.strftime("%Y/%m/%d")
    time_str = now.strftime("%H:%M")

    # 準備新資料
    new_row = [date_str, time_str, new_item, new_amount, new_note]

    # gspread 用 update 改寫該列
    ws.update(f"A{idx+2}:E{idx+2}", [new_row])
    print("已更新為：", new_row)

In [45]:
#請執行本段主程式
while True:
    print("\n=== 簡易記帳選單 ===")
    print("1. 新增記帳")
    print("2. 刪除資料")
    print("3. 修改資料")
    print("4. 退出")

    choice = input("請選擇功能 (1/2/3/4): ").strip()

    if choice == "1":
        add_record(ws)
    elif choice == "2":
        delete_record(ws)
    elif choice == "3":
        modify_record(ws)
    elif choice == "4":
        print("已退出程式。")
        break
    else:
        print("輸入錯誤，請重新輸入。")


=== 簡易記帳選單 ===
1. 新增記帳
2. 刪除資料
3. 修改資料
4. 退出
請選擇功能 (1/2/3/4): 3
目前資料：
   編號          日期     時間   項目     金額   備註
0   0   2025/9/11   8:45   早餐   90.0  NaN
1   1   2025/9/11  13:01   午餐  100.0  NaN
2   2   2025/9/12  15:30   文具   16.0  西卡紙
3   3  2025-09-17  15:46  交通費   15.0   公車
4   4  2025-09-17  15:42   消夜  130.0  NaN
請輸入要修改的編號: 3
原資料： {'日期': '2025-09-17', '時間': '15:46', '項目': '交通費', '金額': 15.0, '備註': '公車'}
項目 [交通費]: 車資
金額 [15.0]: 45
備註 [公車]: 捷運


/tmp/ipython-input-2293967327.py:42: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  ws.update(f"A{idx+2}:E{idx+2}", [new_row])


已更新為： ['2025/09/17', '15:47', '車資', 45.0, '捷運']

=== 簡易記帳選單 ===
1. 新增記帳
2. 刪除資料
3. 修改資料
4. 退出
請選擇功能 (1/2/3/4): 2
目前資料：
   編號          日期     時間  項目     金額   備註
0   0   2025/9/11   8:45  早餐   90.0  NaN
1   1   2025/9/11  13:01  午餐  100.0  NaN
2   2   2025/9/12  15:30  文具   16.0  西卡紙
3   3  2025/09/17  15:47  車資   45.0   捷運
4   4  2025-09-17  15:42  消夜  130.0  NaN
請輸入要刪除的編號: 4
已刪除第 4 筆資料。

=== 簡易記帳選單 ===
1. 新增記帳
2. 刪除資料
3. 修改資料
4. 退出
請選擇功能 (1/2/3/4): 1
請輸入消費項目（例如：午餐、交通）: 買菜
請輸入金額（只接受數字）: 200
是否加入備註？(y/n): 高麗菜
已寫入： ['2025/09/17', '15:48', '買菜', 200.0, '']

=== 簡易記帳選單 ===
1. 新增記帳
2. 刪除資料
3. 修改資料
4. 退出
請選擇功能 (1/2/3/4): 6
輸入錯誤，請重新輸入。

=== 簡易記帳選單 ===
1. 新增記帳
2. 刪除資料
3. 修改資料
4. 退出
請選擇功能 (1/2/3/4): 4
已退出程式。
